# Query Enhancement -  Query Decomposition Techniques

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser

d:\RAG\RAG-Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = TextLoader("langchain_crewai.txt")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap = 50,
    separators = ["\n\n", "\n"," ",""]
)

chunks = splitter.split_documents(documents)

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embedding=embedding_model)

In [5]:
retriever = vector_store.as_retriever(
    search_type ="mmr",
    search_kwargs= {"k":4, "lambda_mult":0.7}
)

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EC85644E90>, search_type='mmr', search_kwargs={'k': 4, 'lambda_mult': 0.7})

In [6]:
llm= init_chat_model("groq:openai/gpt-oss-20b")


In [11]:
decomposition_prompt = PromptTemplate.from_template(
                                                    """
                                                    You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval. Return the answer with only the sub-questions seperated by new-line characters
                                                                                                        
                                                    Questions: "{question}"
                                                                                                        
                                                    Sub-questions:
                                                    """)

decomposition_chain= decomposition_prompt | llm | StrOutputParser()

In [13]:
query = "How does Langchain use memory and agents compared to CrewAI?"
decomposition_question = decomposition_chain.invoke({"question":query})

decomposition_question

"What memory features does LangChain provide?  \nWhat agent capabilities does LangChain support?  \nWhat memory and agent implementations does CrewAI use?  \nHow do LangChain's memory and agent implementations compare to those of CrewAI?"

In [10]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

qa_prompt = PromptTemplate.from_template("""
                                         Use the context below to answer the question.

                                         Context:
                                         {context}

                                         Question: {input}
                                         """)

qa_chain = create_stuff_documents_chain(llm=llm, prompt= qa_prompt)

In [15]:
def query_decomposition_rag_pipeline(user_query: str)-> str:
    sub_queries_text= decomposition_chain.invoke({"question":user_query})
    sub_queries=[q.strip() for q in sub_queries_text.split("\n")]
    answers= []
    for q in sub_queries:
        docs = retriever.invoke(q)
        result = qa_chain.invoke({"input": q, "context": docs})
        answers.append(f"Q: {q}\nA: {result}")

    return "\n\n".join(answers)

In [16]:
query = "How does Langchain use memory and agents compared to CrewAI?"
final_answer = query_decomposition_rag_pipeline(query)
print("Final Answer: \n")
print(final_answer)

Final Answer: 

Q: What mechanisms does Langchain provide for memory management in its agents?
A: **LangChain’s memory‑management tools for agents**

| Feature | What it does | Typical use‑case in an agent |
|--------|--------------|-----------------------------|
| **Memory interfaces** | Abstract base classes (`BaseMemory`) that define the contract for storing and retrieving data. | Lets any agent plug in a custom memory backend without changing its logic. |
| **Built‑in memory implementations** | • **ConversationBufferMemory** – keeps a simple buffer of the last N turns.<br>• **ConversationSummaryMemory** – keeps a running summary of the conversation instead of the full history.<br>• **ConversationTokenBufferMemory** – stores history up to a token budget.<br>• **ConversationEntityMemory** – stores structured entities extracted from the dialogue.<br>• **ConversationBufferWindowMemory** – keeps a sliding window of the most recent turns. | Agents can choose the most appropriate strategy